# Word 2 Vector + + CNN + Logistics Regression

# Word 2 Vector + + CNN + Decison Tree

## Run 1

### Step 1: Word 2 Vec+ CNN

In [ ]:
% time # to see wall time 
# Import required libraries
from google.colab import drive
import pickle
import numpy as np
import pandas as pd

# Mount the Google Drive to access the dataset
drive.mount('/content/drive')

# Load the training data from Google Drive
with open('/content/drive/MyDrive/Ablation/data/X_train.pkl', 'rb') as f:
    datax_train = pickle.load(f)

with open('/content/drive/MyDrive/Ablation/data/Y_train.pkl', 'rb') as f:
    datay_train = pickle.load(f)

# Load the validation data from Google Drive
with open('/content/drive/MyDrive/Ablation/data/X_valid.pkl', 'rb') as f:
    datax_valid = pickle.load(f)

with open('/content/drive/MyDrive/Ablation/data/Y_valid.pkl', 'rb') as f:
    datay_valid = pickle.load(f)


# Load the validatiotestn data from Google Drive
with open('//content/drive/MyDrive/Ablation/data//X_test.pkl', 'rb') as f:
    datax_test = pickle.load(f)
    
with open('//content/drive/MyDrive/Ablation/data//Y_test.pkl', 'rb') as f:
    datay_test = pickle.load(f)

#Code was used from Author Code
def prepare_data(seqs, labels, vocabsize, maxlen=None):
    """Create the matrices from the datasets.
    This function pads each sequence to the same length: the length of the
    longest sequence or maxlen. If maxlen is set, all sequences will be cut
    to this maximum length. This function also swaps the axis.
    
    Args:
        seqs (list): A list of sequences.
        labels (list): A list of labels.
        vocabsize (int): The size of the vocabulary.
        maxlen (int): The maximum length of a sequence (optional).
    
    Returns:
        tuple: A tuple containing the following arrays:
            x (np.array): The input matrix of shape (n_samples, maxlen, vocabsize).
            x_mask (np.array): The input mask matrix of shape (n_samples, maxlen).
            y (np.array): The output matrix of shape (n_samples, maxlen).
            lengths (list): A list of sequence lengths.
            eventLengths (list): A list of event sequence lengths.
    """
    
    # Get the lengths of all sequences
    lengths = [len(s) for s in seqs]

    # Concatenate all visits in each sequence to create event sequences
    eventSeq = []
    for seq in seqs:
        t = []
        for visit in seq:
            t.extend(visit)
        eventSeq.append(t)
    eventLengths = [len(s) for s in eventSeq]

    # If maxlen is set, truncate sequences longer than maxlen and discard
    # sequences that are shorter than maxlen
    if maxlen is not None:
        new_seqs = []
        new_lengths = []
        new_labels = []
        for l, s, la in zip(lengths, seqs, labels):
            if l < maxlen:
                new_seqs.append(s)
                new_lengths.append(l)
                new_labels.append(la)
            else:
                new_seqs.append(s[:maxlen])
                new_lengths.append(maxlen)
                new_labels.append(la[:maxlen])
        lengths = new_lengths
        seqs = new_seqs
        labels = new_labels

        # Return None if there are no sequences left after truncation
        if len(lengths) < 1:
            return None, None, None

    n_samples = len(seqs)
    maxlen = np.max(lengths)

    # Initialize the input matrix, the input mask matrix, and the output matrix
    x = np.zeros((n_samples, maxlen, vocabsize)).astype('int64')
    x_mask = np.zeros((n_samples, maxlen)).astype('float64')
    y = np.zeros((n_samples, maxlen)).astype('int64')

    # Fill the input matrix with the one-hot-encoded events
    for idx, s in enumerate(seqs):
        x_mask[idx, :lengths[idx]] = 1
        for j, sj in enumerate(s):
            for tsj in sj:
                x[idx, j, tsj-1] = 1

    # Fill the output matrix with the labels
    for idx, t in enumerate(labels):
        y[idx,:lengths[idx]] = t

    return x, x_mask, y, lengths, eventLengths



# Prepare training, testing, and validation data
training = prepare_data(datax_train, datay_train, 100, 50)
testing = prepare_data(datax_test, datay_test, 100, 50)
validation = prepare_data(datax_valid, datay_valid, 100, 50)

# Import necessary libraries for machine learning models
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Import necessary libraries for CNN model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


# reshape data for CNN
training[0].shape
len(datay_train[4])

train_X=training[0]
train_X=train_X.reshape(2800,50,100)
train_Y=datay_train

# reshape validation data
valid_X=validation[0]
valid_X=valid_X.reshape(100,50,100)
valid_Y=datay_valid

# reshape test data
test_X=testing[0]
test_X=test_X.reshape(100,50,100)
test_Y=datay_test

# convert train and test labels to numpy arrays
train_Y=np.array(train_Y)
test_Y=np.array(test_Y)

# create CNN model
from tensorflow.keras.models import Model

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(50,100)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(25, activation='softmax'))

# create model for feature extraction
model_features=Model(model.input, model.output)
model_features.summary()

# extract features for train and test data
input_training=model_features.predict(train_X)
print(input_training.shape)
print(train_Y.shape)

input_testing=model_features.predict(test_X)
print(input_testing.shape)
print(test_Y.shape)


import numpy as np

# Convert train_Y to one-hot encoding with padding
train_Y_good = np.zeros([len(train_Y),len(max(train_Y,key = lambda x: len(x)))])
for i,j in enumerate(train_Y):
    train_Y_good[i][0:len(j)] = j

# Convert test_Y to one-hot encoding with padding
test_Y_good = np.zeros([len(test_Y),len(max(test_Y,key = lambda x: len(x)))])
for i,j in enumerate(test_Y):
    test_Y_good[i][0:len(j)] = j

from sklearn.linear_model import LogisticRegression

# Print shape of train_X
train_X[0].shape

# Print first row of train_Y_good
train_Y_good[0]

# Print shape of input_training
input_training.shape

# Print shape of train_Y_good
train_Y_good.shape

# Remove columns from train_Y_good where all elements are zero
train_Y_good = train_Y_good[:,train_Y_good.sum(axis=0)!=0]
train_Y_good.shape

# Remove columns from test_Y_good where all elements are zero
test_Y_good = test_Y_good[:,test_Y_good.sum(axis=0)!=0]
test_Y_good.shape

# Print shape of train_Y_good
train_Y_good.shape

# Slice train_Y_good to keep only first 194 columns
train_Y_good=train_Y_good[:,0:194]

# Print shape of train_Y_good
train_Y_good.shape

# Print shape of test_Y_good
test_Y_good.shape









Mounted at /content/drive


<ipython-input-1-d2202b6f16fd>:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_Y=np.array(train_Y)
<ipython-input-1-d2202b6f16fd>:138: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_Y=np.array(test_Y)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_input (InputLayer)   [(None, 50, 100)]         0         
                                                                 
 conv1d (Conv1D)             (None, 48, 64)            19264     
                                                                 
 conv1d_1 (Conv1D)           (None, 46, 64)            12352     
                                                                 
 dropout (Dropout)           (None, 46, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1472)              0         
                                                             

(100, 194)

The code first mounts the Google Drive to access the data. It then loads the training, validation, and testing data from the pickle files.

The prepare_data() function is then defined, which prepares the input data for the CNN model. This function converts the input sequences to a matrix with each row corresponding to a sequence of the same length. Each row contains the one-hot-encoded representation of the events in the sequence. The labels are also converted to a matrix, where each row corresponds to a sequence of labels.

The code then prepares the training, testing, and validation data by calling the prepare_data() function with the required arguments.

Next, the code imports necessary libraries from scikit-learn for classification tasks such as MultiLabelBinarizer, MultiOutputClassifier, LogisticRegression, SVC, GaussianNB, LinearDiscriminantAnalysis. The code also imports the required libraries from Keras for the CNN model such as Sequential, Dense, Flatten, Dropout, Conv1D, MaxPooling1D.

After that, the code reshapes the training, validation, and testing input data to the required shape for the CNN model. Then it creates a CNN model using the Keras library, with two Conv1D layers, followed by a Dropout layer, MaxPooling1D layer, and two Dense layers with softmax activation.

Finally, it extracts the features of the training data using the model_features object and prints them.

# Step 2 : Logistic Regression

In [ ]:
%time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# Define the estimator and parameters for GridSearchCV
estimator = MultiOutputClassifier(LogisticRegression())
parameters = {
    'estimator__solver': [ 'lbfgs', 'liblinear'],
    'estimator__penalty': ['l1', 'l2', 'elasticnet'],
    'estimator__C': [0.1, 2.0, 10.0]
}

# Define the GridSearchCV object with f1-score as the scoring metric
clf = GridSearchCV(estimator, parameters, scoring='roc_auc',verbose=4,error_score=0)

# Fit the GridSearchCV object to the training data
clf.fit(input_training, train_Y_good)

# Print the best parameters and f1-score
print('Best parameters:', clf.best_params_)
print('Best roc-score:', clf.best_score_)


CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 6.91 µs
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 2/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 3/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 4/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 5/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=lbfgs;, score=0.000 total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 580, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 118, in _average_binary_score
    score[c] = binary_metric(y_true_c, y_score_c, sample_weight=score_w

[CV 1/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=liblinear;, score=0.000 total time=   0.9s
[CV 2/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=liblinear;, score=0.500 total time=   0.9s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 580, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 118, in _average_binary_score
    score[c] = binary_metric(y_true_c, y_score_c, sample_weight=score_w

[CV 3/5] END estimator__C=0.1, estimator__penalty=l1, estimator__solver=liblinear;, score=0.000 total time=   0.9s


...

[CV 3/5] END estimator__C=10.0, estimator__penalty=l2, estimator__solver=liblinear;, score=0.000 total time=   1.0s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 580, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 118, in _average_binary_score
    score[c] = binary_metric(y_true_c, y_score_c, sample_weight=score_w

[CV 4/5] END estimator__C=10.0, estimator__penalty=l2, estimator__solver=liblinear;, score=0.000 total time=   1.0s
[CV 5/5] END estimator__C=10.0, estimator__penalty=l2, estimator__solver=liblinear;, score=0.580 total time=   1.1s
[CV 1/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 2/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 3/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 4/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 5/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=lbfgs;, score=0.000 total time=   0.0s
[CV 1/5] END estimator__C=10.0, estimator__penalty=elasticnet, estimator__solver=liblinear;, score=0.000 total time=   0.0s
[CV 2/5] END estimator__C=10.0, estimator__p

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
45 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/multioutput.py", line 450, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/multioutput.py", line 216, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "/usr/local/lib/python3.10/

Best parameters: {'estimator__C': 10.0, 'estimator__penalty': 'l2', 'estimator__solver': 'lbfgs'}
Best roc-score: 0.2474436753967851


This code is performing hyperparameter tuning for a logistic regression model with multiple outputs using GridSearchCV from scikit-learn. Here is what each part of the code is doing:

%time is a special command in Jupyter Notebook that measures the execution time of the cell.
from sklearn.model_selection import GridSearchCV imports the GridSearchCV class from scikit-learn, which is a tool for performing hyperparameter tuning.
from sklearn.metrics import f1_score imports the f1_score function from scikit-learn, which is a popular evaluation metric for binary classification models.
from sklearn.linear_model import LogisticRegression imports the LogisticRegression class from scikit-learn, which is a classification algorithm that uses logistic regression.
from sklearn.multioutput import MultiOutputClassifier imports the MultiOutputClassifier class from scikit-learn, which is a wrapper that allows fitting multiple classifiers for multiple outputs.
estimator = MultiOutputClassifier(LogisticRegression()) defines a logistic regression model with multiple outputs using MultiOutputClassifier from scikit-learn.
parameters is a dictionary that defines the hyperparameters and their possible values that will be explored during the GridSearchCV process.
clf = GridSearchCV(estimator, parameters, scoring='roc_auc', verbose=4, error_score=0) defines the GridSearchCV object with the logistic regression estimator and the hyperparameters defined in parameters. The scoring parameter specifies the metric that will be used to evaluate the performance of the model, which is roc_auc in this case. The verbose parameter controls the amount of output that is printed during the GridSearchCV process. The error_score parameter determines the value to assign to the score if an error occurs during the model fitting process.
clf.fit(input_training, train_Y_good) fits the GridSearchCV object to the training data input_training and train_Y_good.
print('Best parameters:', clf.best_params_) prints the hyperparameters that resulted in the best performance of the model.
print('Best roc-score:', clf.best_score_) prints the score that resulted from the best hyperparameters found during the GridSearchCV process.

In [ ]:
"""Experiment 3 Word2Vec+CNN+LR"""

# Fit MultiOutputClassifier with LogisticRegression estimator
Model = MultiOutputClassifier(estimator= LogisticRegression(max_iter=500,C=10,penalty='l2',solver='lbfgs')).fit(input_training, train_Y_good)

# Predict test set labels
y_pred=Model.predict(input_testing)

# Initialize empty lists to store performance metrics
PR_AUC=[]
ACC=[]
ROC_AUC=[]

# Loop through each sample in the test set
for i in range(input_testing.shape[0]):
  
  # Reshape the input for the current sample
  inter=input_testing[i].reshape(1,-1)
  
  # Get predicted probabilities for each class label for the current sample
  y_score = Model.predict_proba(inter)
  
  # Get predicted class labels for the current sample
  y_pred=Model.predict(inter)
  
  # Extract predicted probabilities for class label 1 for the current sample
  probabilities=np.concatenate( y_score, axis=0)[:,1]
  
  # Reshape the ground truth labels for the current sample
  train_Y_good[i]=np.array(train_Y_good[i])
  train_Y_good[i]=train_Y_good[i].reshape(1,-1)
  
  # Calculate performance metrics if there is at least one positive class label for the current sample
  if train_Y_good[i].sum()>0:
    auc_roc=roc_auc_score(train_Y_good[i],probabilities)
    average_precision = average_precision_score(train_Y_good[i], probabilities)

  # Set performance metrics to 0 if there are no positive class labels for the current sample
  else:
    auc_roc=0
    average_precision=0
  
  # Calculate accuracy for the current sample
  acc=accuracy_score(train_Y_good[i],y_pred.reshape(-1,1))
  
  # Append performance metrics to respective lists
  PR_AUC.append(average_precision)
  ACC.append(acc)
  ROC_AUC.append(auc_roc)

# Print average performance metrics over all test set samples
print('ACC score is',sum(ACC)/len(ACC))
print('ROC_AUC score is',sum(ROC_AUC)/len(ROC_AUC))
print('PR_AUC score is',sum(PR_AUC)/len(PR_AUC))

# Display CPU time taken to execute the code
%time


ACC score is 0.9192783505154636
ROC_AUC score is 0.7788886904429424
PR_AUC score is 0.2652189037588893
CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 5.96 µs


In [ ]:
%whos # space and memory

Variable                     Type                     Data/Info
---------------------------------------------------------------
ACC                          list                     n=100
Conv1D                       type                     <class 'keras.layers.conv<...>olutional.conv1d.Conv1D'>
Dense                        type                     <class 'keras.layers.core.dense.Dense'>
Dropout                      type                     <class 'keras.layers.regu<...>ization.dropout.Dropout'>
Flatten                      type                     <class 'keras.layers.reshaping.flatten.Flatten'>
GaussianNB                   ABCMeta                  <class 'sklearn.naive_bayes.GaussianNB'>
GridSearchCV                 ABCMeta                  <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
LinearDiscriminantAnalysis   type                     <class 'sklearn.discrimin<...>earDiscriminantAnalysis'>
LogisticRegression           type                     <class 'sklearn.linear_mo<

# Step 3:  Decision Tree Classifier

In [ ]:
%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
# Define the estimator and parameters for GridSearchCV
estimator = MultiOutputClassifier(DecisionTreeClassifier())
parameters = {
    'estimator__max_depth': [10, 50, 100],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# Define the GridSearchCV object with f1-score as the scoring metric
clf = GridSearchCV(estimator, parameters, scoring='roc_auc',error_score=0.5)

# Fit the GridSearchCV object to the training data
clf.fit(input_training, train_Y_good)

# Print the best parameters and f1-score
print('Best parameters:', clf.best_params_)
print('Best roc-score:', clf.best_score_)


CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 5.96 µs


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0.5. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 399, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py", line 580, in roc_auc_score
    return _average_binary_score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_base.py", line 118, in _average_binary_score
    score[c] = binary_metric(y_true_c, y_score_c, sample_weight=score

Best parameters: {'estimator__max_depth': 50, 'estimator__min_samples_leaf': 4, 'estimator__min_samples_split': 2}
Best roc-score: 0.5425513433259975


This code performs a hyperparameter tuning using a Decision Tree Classifier for a multi-output classification task. Here's a breakdown of the code:

%time is a magic command in Jupyter Notebook that measures and prints the execution time of the following code.
The DecisionTreeClassifier from scikit-learn's tree module is imported.
The GridSearchCV class from scikit-learn's model_selection module is imported. This is used to perform hyperparameter tuning by searching for the best combination of hyperparameters.
The f1_score function from scikit-learn's metrics module is imported. This will be used to evaluate the performance of the model during hyperparameter tuning.
The MultiOutputClassifier from scikit-learn's multioutput module is imported. This will be used to create a multi-output version of the DecisionTreeClassifier, which can be used to predict multiple target variables at once.
The estimator variable is set to a multi-output version of DecisionTreeClassifier.
The parameters dictionary is defined to specify the hyperparameters to be tuned. This includes max_depth, min_samples_split, and min_samples_leaf.
The GridSearchCV object is created with the estimator, parameters, and scoring set to 'roc_auc'. This means that the area under the receiver operating characteristic (ROC) curve will be used to evaluate the performance of the model during hyperparameter tuning. The error_score parameter is set to 0.5, which means that any combination of hyperparameters that raises an error during training will receive a score of 0.5.
The GridSearchCV object is fitted to the training data using the fit method.
The best hyperparameters and the best ROC score achieved during hyperparameter tuning are printed using the best_params_ and best_score_ attributes of the GridSearchCV object, respectively.

In [ ]:
"""Ablation Experiment 4: Word2Vec+CNN+DTC"""



# Train the model
Model = MultiOutputClassifier(estimator= DecisionTreeClassifier(max_depth=100,min_samples_leaf=1,min_samples_split=10)).fit(input_training, train_Y_good)

# Make predictions on test data
y_pred=Model.predict(input_testing)

# Compute evaluation metrics
PR_AUC=[]
ACC=[]
ROC_AUC=[]

for i in range(input_testing.shape[0]):
  inter=input_testing[i].reshape(1,-1)
  y_score = Model.predict_proba(inter)
  y_pred=Model.predict(inter)
  probabilities=np.concatenate( y_score, axis=0)[:,1]
  train_Y_good[i]=np.array(train_Y_good[i])
  train_Y_good[i]=train_Y_good[i].reshape(1,-1)
  if train_Y_good[i].sum()>0:
    auc_roc=roc_auc_score(train_Y_good[i],probabilities)
    average_precision = average_precision_score(train_Y_good[i], probabilities)

  else:
    auc_roc=0
    average_precision=0
  acc=accuracy_score(train_Y_good[i],y_pred.reshape(-1,1))
  PR_AUC.append(average_precision)
  ACC.append(acc)
  ROC_AUC.append(auc_roc)

# Print evaluation metrics
print('ACC score is',sum(ACC)/len(ACC))
print('ROC_AUC score is',sum(ROC_AUC)/len(ROC_AUC))
print('PR_AUC score is',sum(PR_AUC)/len(PR_AUC))

# Measure execution time
%time


ACC score is 0.9191237113402058
ROC_AUC score is 0.7482500903647535
PR_AUC score is 0.21909921863710352
CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 6.68 µs


This code calculates evaluation metrics such as accuracy, PR AUC, and ROC AUC for a given model on the testing data.

The input_testing array contains the features for the testing data and the Model is the trained machine learning model. The loop iterates over each instance of the testing data and makes predictions using the trained model.

The predict_proba method of the trained model is used to obtain the predicted probabilities of the target variable for each instance of the testing data. The probabilities are then used to calculate the PR AUC and ROC AUC scores using the average_precision_score and roc_auc_score functions from the sklearn.metrics module. If the true target variable contains at least one positive label, then the PR AUC and ROC AUC scores are calculated, otherwise, they are set to zero.

The accuracy score is calculated using the accuracy_score function from the sklearn.metrics module. The true target variable is compared with the predicted target variable obtained from the predict method of the trained model.

Finally, the calculated evaluation metrics are stored in the PR_AUC, ACC, and ROC_AUC lists, and the average scores are calculated and printed to the console.

In [ ]:
%whos # space and memory

Variable                     Type                     Data/Info
---------------------------------------------------------------
ACC                          list                     n=100
Conv1D                       type                     <class 'keras.layers.conv<...>olutional.conv1d.Conv1D'>
DecisionTreeClassifier       ABCMeta                  <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
Dense                        type                     <class 'keras.layers.core.dense.Dense'>
Dropout                      type                     <class 'keras.layers.regu<...>ization.dropout.Dropout'>
Flatten                      type                     <class 'keras.layers.reshaping.flatten.Flatten'>
GaussianNB                   ABCMeta                  <class 'sklearn.naive_bayes.GaussianNB'>
GridSearchCV                 ABCMeta                  <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
LinearDiscriminantAnalysis   type                     <class 'sklearn.discrimin<